# Complex stationary time series classification

Lead author: Yann Cabanes

# Import the modules

In [1]:
import os
import sys
import warnings

sys.path.append(os.path.dirname(os.getcwd()))
warnings.filterwarnings("ignore")

In [2]:
import geomstats.backend as gs
from geomstats.datasets.prepare_stationary_time_series import *
from geomstats.datasets.simulate_radar_clutter import simulate_clutter
from geomstats.geometry.poincare_complex_disk import PoincareComplexDisk
from geomstats.geometry.positive_reals import PositiveReals
from geomstats.geometry.product_positive_reals_and_poincare_complex_disks import ProductPositiveRealsAndPoincareComplexDisks
from geomstats.learning.mdm import RiemannianMinimumDistanceToMeanClassifier

INFO: Using numpy backend


# Create the dataset

In [3]:
n_cells = 200
n_pulses = 20
spectrum_mean_1 = 0.1
spectrum_mean_2 = - 0.1
variance_1 = 0.05
variance_2 = 0.05

In [4]:
dataset_1, temporal_correlation_matrix_1, spatial_correlation_matrix_1, texture_1 = simulate_clutter(
    n_cells=n_cells, 
    n_pulses = n_pulses, 
    thermal_noise_power=0,
    add_texture=False,
    clutter_power=1,
    spectrum_shape='Gaussian',
    spectrum_mean=spectrum_mean_1,
    spectrum_variance=variance_1,
    spatial_correlation_shape='independent',
    spatial_correlation_coefficient=0.5)

In [5]:
dataset_2, temporal_correlation_matrix_2, spatial_correlation_matrix_2, texture_2 = simulate_clutter(
    n_cells=n_cells, 
    n_pulses = n_pulses, 
    thermal_noise_power=0,
    add_texture=False,
    clutter_power=1,
    spectrum_shape='Gaussian',
    spectrum_mean=spectrum_mean_2,
    spectrum_variance=variance_2,
    spatial_correlation_shape='independent',
    spatial_correlation_coefficient=0.5)

In [6]:
print(dataset_1.shape)

(200, 20)


In [7]:
training_dataset_1 = dataset_1[:100, :]

In [8]:
training_dataset_2 = dataset_2[:100, :]

In [9]:
testing_dataset_1 = dataset_1[100:, :]

In [10]:
testing_dataset_2 = dataset_2[100:, :]

In [11]:
training_dataset = gs.concatenate([training_dataset_1, training_dataset_2], axis=0)

In [12]:
print(training_dataset.shape)

(200, 20)


In [13]:
testing_dataset = gs.concatenate([testing_dataset_1, testing_dataset_2], axis=0)

In [14]:
training_labels = gs.concatenate([gs.zeros(100, dtype=int), gs.ones(100, dtype=int)])

In [15]:
print(training_labels)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [16]:
testing_labels = gs.concatenate([gs.zeros(100, dtype=int), gs.ones(100, dtype=int)])

# Represent the points in the manifold

In [17]:
n_manifolds = 5

In [18]:
training_dataset_in_manifold_1 = burg_regularized(
    training_dataset_1, 
    model_order=4,
    gamma='default')

In [19]:
training_dataset_in_manifold_2 = burg_regularized(
    training_dataset_2, 
    model_order=4,
    gamma='default')

In [20]:
print(training_dataset_in_manifold_1.shape)

(100, 5)


In [21]:
print(training_dataset_in_manifold_1[0, :])


[ 1.19653224+0.j         -0.01651051-0.32141993j  0.12728618-0.11106974j
  0.01489761-0.02959776j  0.00409487+0.11338589j]


In [22]:
testing_dataset_in_manifold_1 = burg_regularized(
    testing_dataset_1, 
    model_order=4,
    gamma='default')

In [23]:
testing_dataset_in_manifold_2 = burg_regularized(
    testing_dataset_2, 
    model_order=4,
    gamma='default')

In [24]:
training_dataset_in_manifold = gs.concatenate(
    [training_dataset_in_manifold_1, training_dataset_in_manifold_2], 
    axis=0)

In [25]:
testing_dataset_in_manifold = gs.concatenate(
    [testing_dataset_in_manifold_1, testing_dataset_in_manifold_2], 
    axis=0)

# Check if the data belong to the right space

In [26]:
product_positive_reals_and_poincare_complex_disks = ProductPositiveRealsAndPoincareComplexDisks(
    n_manifolds=n_manifolds)

In [27]:
belongs = gs.all(product_positive_reals_and_poincare_complex_disks.belongs(
    training_dataset_in_manifold))

In [28]:
print(belongs)

True


In [29]:
belongs = gs.all(product_positive_reals_and_poincare_complex_disks.belongs(
    testing_dataset_in_manifold))

In [30]:
print(belongs)

True


# Visualize the data

# Minimum distance to mean

In [31]:
mdm = RiemannianMinimumDistanceToMeanClassifier(
    riemannian_metric=product_positive_reals_and_poincare_complex_disks.metric, 
    n_classes=2, 
    point_type="vector")

In [32]:
mdm.fit(X=training_dataset_in_manifold, y=training_labels)

In [33]:
predicted_labels = mdm.predict(X=testing_dataset_in_manifold)

In [34]:
print(predicted_labels)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0
 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1]


In [35]:
score = mdm.score(training_dataset_in_manifold, testing_labels)

In [36]:
print(score)

0.915
